In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import re

def separate_integers_letters(string):
    integers = ''.join(re.findall(r'\d+', string))
    letters = ''.join(re.findall(r'[a-zA-Z]+', string))
    return integers, letters

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def add_labels(df):
  label_encoder = LabelEncoder()

  for index, row in df.iterrows():
    station_name_parts = row['station_name'].split('/')
    position_dynamique = station_name_parts[2]
    position_dynamique_parts = position_dynamique.split('_')
    _, position = separate_integers_letters(position_dynamique_parts[1])
    dynamique = position_dynamique_parts[2]

    df.loc[index, 'position'] = position
    df.loc[index, 'dynamique'] = dynamique

  df['position'] = label_encoder.fit_transform(df['position'])
  df['dynamique'] = label_encoder.fit_transform(df['dynamique'])

  df['label'] = df['position'].astype(str) + df['dynamique'].astype(str)
  df = df.drop(['position', 'dynamique'], axis=1)

  return df

In [5]:
from sklearn.metrics import accuracy_score

def custom_cross_validation(model, X_train, y_train, X_test, y_test):
    # Entraîner le modèle sur l'ensemble de données d'entraînement
    model.fit(X_train, y_train)

    # Prédire les étiquettes sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Calculer la précision du modèle
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [6]:
import pandas as pd
import os
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Chemin vers le répertoire contenant les dossiers de données sur Google Drive
data_directory = "/content/gdrive/MyDrive/ListenToTheWild/day_temporal"

directory = "day_temporal"

# Charger les données d'entraînement
train_data = pd.read_csv(os.path.join(data_directory, f"{directory}_train.csv"))
nb_rows_train = train_data.shape[0]
# Ajouter les labels
train_data_with_labels = add_labels(train_data)
# Supprimer les colonnes "file_name" et "station_name"
train_data_with_labels = train_data_with_labels.drop(['file_name', 'station_name'], axis=1)
# Pour supprimer les NaN
train_data_with_labels = train_data_with_labels.drop(['Ht', 'ACTtMean', 'TFSD'], axis=1)
# Créer X et y
X_train = train_data_with_labels.drop(columns=['label'])  # attributs
y_train = train_data_with_labels['label']  # étiquettes

# Charger les données de test
test_data = pd.read_csv(os.path.join(data_directory, f"{directory}_test.csv"))
nb_rows_test = test_data.shape[0]
# Ajouter les labels
test_data_with_labels = add_labels(test_data)
# Supprimer les colonnes "file_name" et "station_name"
test_data_with_labels = test_data_with_labels.drop(['file_name', 'station_name'], axis=1)
# Pour supprimer les NaN
test_data_with_labels = test_data_with_labels.drop(['Ht', 'ACTtMean', 'TFSD'], axis=1)
# Créer X et y
X_test = test_data_with_labels.drop(columns=['label'])  # attributs
y_test = test_data_with_labels['label']  # étiquettes

# Créer les modèles
rf_model = RandomForestClassifier()
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500)
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

# Effectuer les validations croisées personnalisées
rf_accuracy = custom_cross_validation(rf_model, X_train, y_train, X_test, y_test)
mlp_accuracy = custom_cross_validation(mlp_model, X_train, y_train, X_test, y_test)
gb_accuracy = custom_cross_validation(gb_model, X_train, y_train, X_test, y_test)

# Calculer la moyenne des précisions
mean_accuracy = (rf_accuracy + mlp_accuracy + gb_accuracy)/3

# Calculer les pourcentages des ensembles
per_train_set = nb_rows_train / (nb_rows_train + nb_rows_test)
per_test_set = nb_rows_test / (nb_rows_train + nb_rows_test)

# Afficher les précisions sur l'ensemble de test
print(f"Dossier {directory} (train : {round(per_train_set*100, 2)}% / test : {round(per_test_set*100, 2)}%) :")
print(f"Précision sur l'ensemble de test avec RF : {round(rf_accuracy, 2)}")
print(f"Précision sur l'ensemble de test avec MLP : {round(mlp_accuracy, 2)}")
print(f"Précision sur l'ensemble de test avec GB : {round(gb_accuracy, 2)}")
print(f"Moyenne des précisions : {round(mean_accuracy, 2)}")
print()

Dossier day_temporal (train : 80.02% / test : 19.98%) :
Précision sur l'ensemble de test avec RF : 0.37
Précision sur l'ensemble de test avec MLP : 0.23
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.31

